# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Volumes/GoogleDrive/My Drive/Study/DataEngineer_NanoDegree/4.NoSQLDataModels/data_modeling_with_apache_cassandra_project_starter


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance in local machine 

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = {
            'class': 'SimpleStrategy',
            'replication_factor': 1}
        """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Create a table to give the following info for sessionId = 338, and itemInSession = 4
## - artist
## - song title
## - song's length in the music app history
## The query used sessionId and itemInSession as PRIMARY KEY to make unique pair 

query_1 = "CREATE TABLE IF NOT EXISTS music_library"
query_1 = query_1 + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query_1)
except Exception as e:
    print(e)

In [9]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements
        query = "INSERT INTO music_library (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assigns column element to execute INSERT query
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## SELECT statement to verify the data was entered into the table
query = ("""
    SELECT artist, song, length
    FROM music_library 
    WHERE sessionId = 338 
    AND itemInSession = 4
""")

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2: Create a table to give the following info for userid = 10, sessionid = 182
## - name of artist
## - song (sorted by itemInSession)
## - user (first and last name)
## The query used userId and sessionId as composite partition key to store the paired columns. With itemInSession as clustering columns, PRIMARY KEY can be unique key

query_2 = "CREATE TABLE IF NOT EXISTS user_library"
query_2 = query_2 + "(artist text, song text, firstName text, lastName text, userId int, sessionId int, itemInSession int, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query_2)
except Exception as e:
    print(e)

# Set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements
        query = "INSERT INTO user_library (artist, song, firstName, lastName, userId, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assigns column element to execute INSERT query
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

## SELECT statements
query = (
    """
    SELECT artist, song, firstName, lastName
    FROM user_library
    WHERE userId = 10
    AND sessionId = 182
    """
    )
    
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)
                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [12]:
## Query 3: Create a table to give the following info for song 'All Hands Against His Own'
## - every user name (first and last)
## The query used song and userId as PRIMARY KEY to make them unique key
query_3 = "CREATE TABLE IF NOT EXISTS music_history"
query_3 = query_3 + "(firstName varchar, lastName varchar, song varchar, userId int, PRIMARY KEY (song, userId))"

try:
    session.execute(query_3)
except Exception as e:
    print(e)

# Set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements
        query = "INSERT INTO music_history (firstName, lastName, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assigns column element to execute INSERT query
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

## SELECT statements
query = (
    """
    SELECT firstName, lastName, song
    FROM music_history
    WHERE song = 'All Hands Against His Own'
    """
    )
    
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)
                    
                    

Row(firstname='Jacqueline', lastname='Lynch', song='All Hands Against His Own')
Row(firstname='Tegan', lastname='Levine', song='All Hands Against His Own')
Row(firstname='Sara', lastname='Johnson', song='All Hands Against His Own')


### Drop the tables before closing out the sessions

In [13]:
## Drop the table before closing out the sessions
query_1 = "DROP TABLE music_library"
query_2 = "DROP TABLE user_library"
query_3 = "DROP TABLE music_history"

queries = [query_1, query_2, query_3]

for query in queries:
    try:
        rows = session.execute (query)
    except Exception as e:
        print (e)

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()